In [1]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os as os
import nrrd as reader
import tensorflow as tf
import cv2 
import pandas as pd
from skimage import morphology
from skimage.measure import label
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters

j = 5

In [2]:
def join(one, two):
    return os.path.join(one, two)

In [3]:
def check_path(path):
    count=0
    for files in sorted(os.listdir(path)):
        image_path = os.path.join(path,files)
        count+=1
    return count

In [7]:
#(patient count* 15) check_path function will return total patient count 
folder_path = join(os.getcwd(), "Incoming Annotations")
length = check_path(folder_path)
inputs = np.zeros((length*15,256,256, 1))
outputs = np.zeros((length*15,256,256 , 9))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jasontsang/Desktop/RA-F-ER/JuypterNotebook/Incoming Annotations'

In [ ]:
#functions for inhomogeneity correction 
def correct_roi(image):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    return image_c

In [ ]:
def dcm_to_np(path):
    global dic_count
    try:
        slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path)
        image = sitk.ReadImage(slice_filenames)
        for x in range(15):
            current_slice = image[:, :, x] #************************
            current_array = sitk.GetArrayFromImage(current_slice)
           # Ivol=sitk.GetArrayFromImage(inputs[:, :, :])
            split_array = current_array[:,:-256]
            inputs[dic_count] = split_array[:, :, np.newaxis]
            dic_count+=1
        print(dic_count)
    except Exception as e:
        print(e)
#         print("Problem with this folder -> " + path)

In [ ]:
def nrrd_to_np(meta_object):
    global ann_count
    try:
        data = sitk.GetArrayFromImage(meta_object)
        for p in range(15):
            outputs[ann_count] = np.expand_dims(np.where(data[p, :, :256] == 2, 1, 0), axis=2)
            ann_count+=1
        print(ann_count)
    except Exception as e:
        print(e)

In [ ]:
dic_count = 0
ann_count = 0
try:
    for patient in sorted(os.listdir(folder_path)):
            patient_path = join(folder_path, patient)
            nrrd_folder = join(patient_path, "NRRD")
            dcm_to_np(patient_path)
            for nrrd in os.listdir(nrrd_folder):
                if(".nrrd" in nrrd):
                    segmentation = sitk.ReadImage(join(nrrd_folder, nrrd))
                    nrrd_to_np(segmentation)
                    if(".DS_Store" in nrrd):
                        pass
except Exception as e:
    print(e)


In [ ]:
#opencv + numpy normalization 
for x in range (j):    
    temp = inputs[x].reshape(256, 256)
    temp1= correct_roi(temp)
    np.linalg.norm(temp1)
    inputs[x]=temp1[:, :, np.newaxis]
    print(x)

In [5]:
imageI=inputs[:, :, :, 0] #Inputs Variable 
ArrayI = np.zeros((1755, 256, 256, 1)) 
for x in range (1755): 
    for i in range (9):
        ArrayI[x,:,:,0]+=outputs[x,:,:,i] #Merged MSK Mask
ArrayII=ArrayI[:, :, :, 0] # Get rid of [:,:,:,1]
INVArrayI=1-ArrayII #Invered Mask
INVArrayII=INVArrayI*imageI #Mask multipled with array
INVArrayIII=np.where(INVArrayII > 80, 1 , 0) #Get rid of magnetic background noise
ArrayIII = INVArrayIII+ArrayII #Add both the merged msk and inv msk together
FINALArray=ArrayIII*inputs1 #Multiply new mask with original image
for i in range (2):
    plt.imshow(FINALArray[i],cmap=plt.cm.bone,vmin=0,vmax=255)
    plt.show()

NameError: name 'inputs' is not defined